In [ ]:
import sys

sys.path.append("..")
from common_utils import set_data_home

set_data_home("~/datasets")
from common_utils import DATA_HOME, join
from lstm.sales_data import Sales_Dataset
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
SALE_HOME = join(DATA_HOME, "sales_data")
MODEL_HOME = join(DATA_HOME, "sale_model")

In [ ]:
I, H, B = 72, 34, 6
HEAD = 6

sd = Sales_Dataset(SALE_HOME, seq_len=500, is_train=False, device=device)
# model.load_state_dict(torch.load("./sales_model_600.pth"))
sd[0][0].shape, sd[1][0].shape, sd[2][0].shape

In [ ]:
from torch.nn import Transformer, Linear


class MyTransformer(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.trans = Transformer(d_model=I, nhead=HEAD, batch_first=True).cuda()
        self.linear = Linear(I, H)

    def forward(self, src, tgt):
        return self.linear(self.trans(src, tgt))

### returns to sales

In [ ]:
def ret_2_sale(data: torch.Tensor, base_sales: torch.Tensor):
    for i, curr in enumerate(data):
        base_sales = 10**curr * base_sales
        data[i] = base_sales

In [ ]:
import pandas as pd

INFERED_DAYS = 16

model = MyTransformer().cuda()
sales = pd.read_csv(join(SALE_HOME, "test.csv"), index_col=False)
sales["sales"] = 0.0
ans_df = pd.DataFrame()

for X, store_id, lastday in sd:
    X = X.cuda()
    for i in range(INFERED_DAYS):
        yhat = model(X, X)
    print(f"store_id: {store_id}, date: {lastday} infered")

### Transform returns to sales

In [ ]:
from datetime import timedelta
import pandas as pd

sales.sort_values(["store_nbr", "family", "date"], inplace=True)
# sd.TR = pd.read_csv(join(SALE_HOME, "TR.csv"), index_col=False)
sd.TR = sd.TR.reset_index().sort_values(["store_nbr", "family", "date"])
base_sales = sd.base_sales


for s in sd.ids:
    for f in sd.families:
        start_date, end_date = sd.train_max_date + timedelta(days=1), sd.test_max_date
        prev_sale = base_sales.loc[
            (base_sales.store_nbr == s) & (base_sales.family == f)
        ].sales
        for d in pd.date_range(start_date, end_date):
            ret = sd.TR.loc[
                (sd.TR.store_nbr == s) & (sd.TR.family == f) & (sd.TR.date == d)
            ].sales
            curr_sale = 10 ** ret.item() * prev_sale.item()
            sales.loc[
                (sales.store_nbr == s)
                & (sales.family == f)
                & (sales.date == d.strftime("%Y-%m-%d")),
                "sales",
            ] = curr_sale
        print(f"store {s}, family {f} inferred")


base_sales, sd.test_max_date - sd.train_max_date

### output the answer

In [ ]:
sales.drop(columns=["store_nbr", "date", "family", "onpromotion"]).sort_values(
    "id"
).to_csv("answer.csv", index=False)